In [0]:
# 'WB', 'DE', 'MH', 'AP', 'UP', 'PU', 'KA', 'GU', 'RA', 'MP', 'OR', 'TN', 'BI', 'CH', 'HA'
# all_state = ['WB', 'MH', 'AP','UP', 'PU', 'KA', 'GU', 'RA', 'MP', 'OR', 'BI', 'CH','TN']

all_state = ['WB','DE', 'MH', 'AP', 'UP', 'PU', 'KA', 'GU', 'RA', 'MP', 'OR', 'BI', 'CH','HA','TN']
sta_len = len(all_state) 

In [0]:
import pandas as pd
import numpy  as np
# all_state=['HA']

In [3]:
##### reading the file ##########################################################
for sta in range(sta_len):
  
  ##### RAW data ######################################################################################
  coke_raw = pd.read_excel('CCI_Juices_consolidated.xlsx',sheet_name=all_state[sta])    #### load raw data here
  dt_raw   = coke_raw.copy()
  dt_raw   = dt_raw[dt_raw['Date'] >='2015-01-01'].reset_index(drop=True)  ## date change according to model, format:- YYYY-MM-DD
  power_bi = dt_raw.copy()
  power_bi['Category'] = 'Juices'                      # enter category name as needed for power BI
  ##### transformed Data ###############################################################################
  coke_ols = pd.read_excel('Juices_Ensemble_Data.xlsx',sheet_name=all_state[sta])   #### load transformed data here
  dt_ols   = coke_ols.copy()
  dt_ols   = dt_ols[dt_ols['Date'] >='2015-01-01'].reset_index(drop=True)  ## date change according to model, format:- YYYY-MM-DD
  ##### coefficient data ###############################################################################
  coef_ols = pd.read_excel('coeff_Juices.xlsx', sheet_name =all_state[sta])  #### coefficient data here 
  dt_coef  = coef_ols.copy() 
  ######################################################################################################
  seed     = pd.read_excel("Seed_results_Juices.xlsx",sheet_name=all_state[sta])  
  rf_seed  = seed.at[0,'RF']               # rf  seed
  ann_seed = seed.at[0,'ANN']              # ann seed          
  ############################### Dynamic Coefficients ################################################

  ##########################################################################################################
  date1 = '2019-01-01'       # date from which dynamic callculations be calculated, format = 'YYYY-MM-DD'
  date2 = '2018-01-01'       # year back as that of 'date1' 

  due_to = 4                 # mention number of variables(features no. for elasticity calculation) and used in column selection in Due-to
  trans  = [0,0,1,0]         # this array is for log transformation, enter for main features for which we calculate elasticity
  
  
  ##############################################################################################################################################
  ##############################################################################################################################################
  ####### train and test data for rf+ann model ###########################################
  z = coke_ols['VolSales'].dropna().count() - 1
  y = z - 3
  ######## predicted ols column selection ###########################################
  list_ols  = list(coke_ols.columns.values) 

  col_names = list_ols[2:-1]             # Enter column number here and it should be features
  col_count = len(col_names)

  ################################################################# OLS:- y = mx + c --- code part
  Y = coke_ols['VolSales']

  xy = 0
  for i in range(col_count):
    xy += coef_ols['Estimate'][i+1] * coke_ols[coke_ols.columns[i+2]]

  xy = coef_ols['Estimate'][0] + xy


  ols_mape = np.mean(np.abs ( (Y - xy)/Y ) * 100)          
  print("MAPE OLS:",abs(ols_mape))

  coke_ols['OLS_Predicted'] = xy   
  #######################################################################################################
  ############################# RF + ANN model ##########################################################
  #######################################################################################################
  states_name = coke_ols['States'].unique()
  r2_mape = []

  for i in range(len(states_name)):
      metric_train=[]
      metric_test=[]
      state = states_name[i]
      print('------------------------------------------------------------------------------------------------------')
      print('Processing for state: ',state)
      pun_coke  = coke_ols[coke_ols['States'] == state]   # Give your state name here
      pun_coke  = pun_coke.reset_index( )
      #'Market',add below for 7 new states
      pun_coke2 = pun_coke.drop(['States','index'],axis=1)  
      x=0
      # y=56                                                                        
      # z=59                                                                        
      train_1 = pun_coke2.iloc[x:y, :]
      test_1  = pun_coke2.iloc[y:z, :]
      forecast_train = pun_coke2.iloc[x:z, :]
      forecast_date  = pun_coke2.iloc[z:, :]
      train = train_1.drop(['Date'],axis=1)
      test  = test_1.drop(['Date'],axis=1)
      metric_train.append(str((train.columns).values.tolist()))
      metric_test.append(str((train.columns).values.tolist()))
      metric_train.append('TRAIN')
      metric_test.append('TEST')
      
      train_target  = train[['VolSales']]
      train_feature = train.drop(['VolSales'], axis=1)
      test_target   = test[['VolSales']]
      test_feature  = test.drop(['VolSales'], axis=1)

      
      print(train_feature.shape)
      print(train_target.shape)
      print(test_feature.shape)
      print(test_target.shape)
      
      ################################## Modelling #######################################################
      
      def mean_absolute_percentage_error(y_true, y_pred): 
          y_true, y_pred = np.array(y_true), np.array(y_pred)
          return np.mean(np.abs((y_true - y_pred) / y_true)*100)
      
  ##################################
  ###  
  ################################## Random Forest ###########################################
      #n_estimators=10,random_state=42
      #n_estimators=20,max_depth=5,min_samples_split=5,min_samples_leaf=1,
      from sklearn.ensemble import RandomForestRegressor
      rf = RandomForestRegressor(random_state=rf_seed)    
      rf.fit(train_feature,train_target)
      Train_pred_rf = rf.predict(train_feature)
      Test_pred_rf  = rf.predict(test_feature)
      rf.fit(pun_coke2.iloc[x:z, :].drop(['VolSales','Date'], axis=1),pun_coke2.iloc[x:z, :]['VolSales'])
      pred_rf = rf.predict(pun_coke2.iloc[z:, :].drop(['VolSales','Date'], axis=1))
      
      actual_test=list(test_target['VolSales'])
      actual_train=list(train_target['VolSales'])

      Train_Results=pd.DataFrame(data=actual_train, columns=['Train_Actual'])
      Train_Results.insert(loc=1, column='Train_Pred_RF', value=Train_pred_rf)
      Test_Results=pd.DataFrame(data=actual_test, columns=['Test_Actual'])
      Test_Results.insert(loc=1, column='Test_Pred_RF', value=Test_pred_rf)
      

      print("                           R_square for training data:",rf.score(train_feature,train_target))
      print("                           R_square for testing data:",rf.score(test_feature,test_target))
      print("                           MAPE for training data:",mean_absolute_percentage_error(train_target['VolSales'], Train_pred_rf))
      print("                           MAPE for testing data:",mean_absolute_percentage_error(test_target['VolSales'], Test_pred_rf))
      print(rf.feature_importances_)
      fi=pd.DataFrame(rf.feature_importances_,index=train_feature.columns)
      # print(fi)
      r2_train=rf.score(train_feature,train_target)
      r2_test=rf.score(test_feature,test_target)
      mape_train=mean_absolute_percentage_error(train_target['VolSales'], Train_pred_rf)
      mape_test=mean_absolute_percentage_error(test_target['VolSales'], Test_pred_rf)
      

    
      metric_train.append(r2_train)
      metric_train.append(mape_train)
      metric_test.append(r2_test)
      metric_test.append(mape_test)

      rf_mape = (((mape_train)*y)+((mape_test)*(z-y)))/(z+1)
      print("RF mape:-",rf_mape)
      
      ################################## ANN ##############################################
      from sklearn.metrics import r2_score
      from sklearn.neural_network import MLPRegressor

      model_ann = MLPRegressor(hidden_layer_sizes = (256),max_iter=50, batch_size=1,verbose=0,random_state=ann_seed)
      print(model_ann.get_params(deep=True))
      
      model_ann.fit(train_feature, train_target)

      
      model_ann.fit(pun_coke2.iloc[x:z, :].drop(['VolSales','Date'], axis=1),pun_coke2.iloc[x:z, :]['VolSales'])
      pred_ann = model_ann.predict(pun_coke2.iloc[z:, :].drop(['VolSales','Date'], axis=1))
      
      Train_pred_ann=model_ann.predict(train_feature)
      Test_pred_ann=model_ann.predict(test_feature)
      
      Train_Results.insert(loc=2, column='Train_Pred_ANN', value=Train_pred_ann)
      Test_Results.insert(loc=2, column='Test_Pred_ANN', value=Test_pred_ann)
      
      
      print("                R_square for training data:",r2_score(train_target['VolSales'], Train_pred_ann))
      print("                R_square for testing data:",r2_score(test_target['VolSales'], Test_pred_ann))
      print("                MAPE for training data:",mean_absolute_percentage_error(train_target['VolSales'], Train_pred_ann))
      print("                MAPE for testing data:",mean_absolute_percentage_error(test_target['VolSales'], Test_pred_ann))
      
      
      mape_train=mean_absolute_percentage_error(train_target['VolSales'], Train_pred_ann)
      mape_test=mean_absolute_percentage_error(test_target['VolSales'], Test_pred_ann)

      ann_mape = (((mape_train)*y)+((mape_test)*(z-y)))/(z+1)
      print("ANN mape:-",ann_mape)
  #############################################################################################################################
  #########################################################
  estimatorts_train    = [Train_pred_rf, Train_pred_ann]
  estimatorts_test     = [Test_pred_rf, Test_pred_ann]
  estimatorts_forecast = [pred_rf,pred_ann]

  estimatorts_train    = pd.DataFrame(estimatorts_train).T
  estimatorts_test     = pd.DataFrame(estimatorts_test).T
  estimatorts_forecast = pd.DataFrame(estimatorts_forecast).T

  estimatorts=estimatorts_train.append(estimatorts_test)
  estimatorts=estimatorts.append(estimatorts_forecast)

  estimatorts = estimatorts.rename(columns={0: 'Pred_RF',1:'Pred_ANN'})

  estimatorts=estimatorts.reset_index(drop=True)
  estimatorts['Pred_OLS'] = coke_ols['OLS_Predicted']
  #####################################################################################################
  #################################################################
  ###################################################################################
  w1=(((ols_mape+rf_mape+ann_mape)**2)/(ols_mape)**2)
  w2=(((ols_mape+rf_mape+ann_mape)**2)/(rf_mape)**2)
  w3=(((ols_mape+rf_mape+ann_mape)**2)/(ann_mape)**2)

  w11=w1/(w1+w2+w3)
  w22=w2/(w1+w2+w3)
  w33=w3/(w1+w2+w3)


  estimatorts["Ensemble"] = (w11*estimatorts["Pred_OLS"]) + (w22*estimatorts["Pred_RF"]) + (w33*estimatorts["Pred_ANN"])
  ###########################################################################
  e=estimatorts["Ensemble"]                       
      
  estimatorts = estimatorts.reset_index(drop=True)
  estimatorts["Ensemble"] = estimatorts["Ensemble"].reset_index(drop=True)
  coke_ols['Ensemble']    = estimatorts["Ensemble"]

  e = e.reset_index(drop=True)
  Y = Y.reset_index(drop=True)

  Ensemble_MAPE=np.mean(np.abs((Y-e)/Y)*100)
  print("MAPE Ensemble:",abs(Ensemble_MAPE))
  #######################################################################################################################

  #################### part of DUE_TO ###################################################################################

  #######################################################################################################################

  list_df      = list(dt_raw.columns.values)    # columns number that covers raw data's features
  dt_col_names = list_df[2:due_to+2]   
  dt_col_count = len(dt_col_names)  
  due_to_varb  = len(dt_col_names)              # features being used in Due_tos for loop       
  sale_ind     = pd.DatetimeIndex(dt_raw['Date']).year.nunique()  # no. of times Sales Index calculation should repeat
                                                                                  
  ####################### Due-To calculations #######################################################################
  ###################################################################################################################

  ####################### Due-To calculations #######################################################################

  Due_to = pd.DataFrame()
  ##### Difference YOY  ###########################################################################################################
  for i in range(due_to_varb):
    Due_to[dt_col_names[i]+'_Diff_YOY'] = dt_raw[dt_col_names[i]].diff(periods = 12)
  ##### Difference MOM  ########################################################################################################### 
  for i in range(due_to_varb):
    Due_to[dt_col_names[i]+'_Diff_MOM'] = dt_raw[dt_col_names[i]].diff(periods = 1)
  ##### DT of variables  ##########################################################################################################
  for i in range(due_to_varb):
    Due_to['DT_'+dt_col_names[i]] = (dt_coef['Estimate'][i+1] * dt_ols[dt_ols.columns[i+2]]) - (dt_coef['Estimate'][i+1] * dt_ols[dt_ols.columns[i+2]].shift(12))
    Due_to['DT_'+dt_col_names[i]].fillna(0, inplace=True)
  ##### % DT variables  ############################################################################################################
  for i in range(due_to_varb):
    Due_to['% DT_'+dt_col_names[i]] = ((Due_to['DT_'+dt_col_names[i]]).to_numpy() / (dt_ols['VolSales']).shift(12)) * 100
  #### DT sales index  #############################################################################################################
  import datetime
  dt_raw['month']       = pd.DatetimeIndex(dt_raw['Date']).month
  sales_index           = dt_raw[['Date','month','VolSales']]
  x = sales_index.groupby(['month']).mean()
  dt_raw['Sales']       = pd.DataFrame(np.tile((x['VolSales']/(sales_index['VolSales']).mean()),sale_ind))  
  dt_raw['Sales_Index'] = dt_raw['Sales'].shift(12)
  dt_raw.drop(['Sales'], axis=1, inplace=True)

  for i in range(due_to_varb):
    Due_to['DT_'+dt_col_names[i]+'_using_sales_index'] = dt_raw['Sales_Index'] * Due_to['DT_'+dt_col_names[i]]
    Due_to['DT_'+dt_col_names[i]+'_using_sales_index'].fillna(0, inplace=True)

  for i in range(1):
    Due_to['DT Weather Sales Index'] = Due_to['DT_'+dt_col_names[i]+'_using_sales_index'] + Due_to['DT_'+dt_col_names[i+1]+'_using_sales_index'] 
  ##### DT % sales index  #############################################################################################################
  for i in range(due_to_varb):
    Due_to['DT_%'+dt_col_names[i]+'_sales_index'] = ((Due_to['DT_'+dt_col_names[i]+'_using_sales_index']).to_numpy() / (dt_ols['VolSales']).shift(12)) * 100

  for i in range(1):
    Due_to['DT_%Weather_Sales_Index'] = Due_to['DT_%'+dt_col_names[i]+'_sales_index'] + Due_to['DT_%'+dt_col_names[i+1]+'_sales_index']
  ##### DT temp MOM  ##################################################################################################################
  for i in range(due_to_varb):
    Due_to['DT_'+dt_col_names[i]+'_MOM'] = Due_to['DT_'+dt_col_names[i]+'_using_sales_index'].diff(periods=1)

  Due_to['DT_Weather_MOM'] = Due_to['DT Weather Sales Index'].diff(periods=1)
  ##### DT ctualised rolling sums ##################################################################################################################################
  Due_to['actualized']=coke_ols.apply(
	lambda row: row['VolSales'] if ~np.isnan(row['VolSales']) else row['Ensemble'],axis=1)

  Due_to['actualized_3mm'] = Due_to['actualized'].rolling(3).sum().fillna(0)
  Due_to['actualized_6mm'] = Due_to['actualized'].rolling(6).sum().fillna(0)
  #######################################################################################################################################
  Due_to['3MMT_VolSales'] = coke_ols['VolSales'].rolling(3).sum()
  Due_to['3MMT_Ensemble'] = coke_ols['Ensemble'].rolling(3).sum()
  Due_to['3MMT_VolSales'] = coke_ols['VolSales'].rolling(3).sum()
  Due_to['6MMT_Ensemble'] = coke_ols['Ensemble'].rolling(6).sum()
  Due_to['6MMT_VolSales'] = coke_ols['VolSales'].rolling(6).sum()
  Due_to['3MMT_Category'] = dt_raw.iloc[:, [6]].rolling(3).sum()
  Due_to['6MMT_Category'] = dt_raw.iloc[:, [6]].rolling(6).sum()
  intial_rolling = coke_ols.VolSales.rolling(3-1).sum()
  intial_rolling = intial_rolling.shift(1)
  list1 = coke_ols.Ensemble[2:]
  Due_to['3MMT_forecast'] = intial_rolling+list1
  Due_to['Error']=(((Due_to['3MMT_VolSales'].to_numpy())/Due_to['3MMT_forecast'])-1)*100



  ########################### power Bi DT input--part ################################################################

  ########### Diff YOY #####################################################################################
  for i in range(due_to_varb):
    power_bi[dt_col_names[i]+'_Diff_YOY'] = Due_to[dt_col_names[i]+'_Diff_YOY']
  ########### Diff MOM #####################################################################################
  for i in range(due_to_varb):
    power_bi[dt_col_names[i]+'_Diff_MOM'] = Due_to[dt_col_names[i]+'_Diff_MOM']
  ########### DT_using sales index #########################################################################
  for i in range(due_to_varb):
    power_bi['DT_'+dt_col_names[i]+'_using_sales_index'] = Due_to['DT_'+dt_col_names[i]+'_using_sales_index']
  # power_bi['DT Weather Sales Index'] = Due_to['DT Weather Sales Index']
  ########### DT_MOM #######################################################################################
  for i in range(due_to_varb):
    power_bi['DT_'+dt_col_names[i]+'_MOM'] = Due_to['DT_'+dt_col_names[i]+'_MOM']
  # power_bi['DT_Weather_MOM'] = Due_to['DT_Weather_MOM']
  ########### DT_% sales index #############################################################################
  for i in range(due_to_varb):
    power_bi['DT_%'+dt_col_names[i]+'_sales_index'] = Due_to['DT_%'+dt_col_names[i]+'_sales_index']
  # power_bi['DT_%Weather_Sales_Index'] = Due_to['DT_%Weather_Sales_Index']
  ##########################################################################################################
  power_bi['Ensemble'] = coke_ols['Ensemble']
  power_bi['Actualised']     = Due_to['actualized']
  power_bi['Actualised_3MMT'] = Due_to['actualized_3mm']
  power_bi['Actualised_6MMT'] = Due_to['actualized_6mm']
  power_bi['3MMT_Ensemble'] = coke_ols['Ensemble'].rolling(3).sum()
  power_bi['3MMT_VolSales'] = coke_ols['VolSales'].rolling(3).sum()
  power_bi['6MMT_Ensemble'] = coke_ols['Ensemble'].rolling(6).sum()
  power_bi['6MMT_VolSales'] = coke_ols['VolSales'].rolling(6).sum()
  power_bi['3MMT_Category'] = dt_raw.iloc[:, [6]].rolling(3).sum()
  power_bi['6MMT_Category'] = dt_raw.iloc[:, [6]].rolling(6).sum()
  power_bi['3MMT_forecast'] = Due_to['3MMT_forecast']
  power_bi['Error'] = Due_to['Error']
  
  ##########################################################################################################
  for i in range(due_to_varb):
    power_bi[dt_col_names[i]+'3MMT'] = dt_raw[dt_col_names[i]].rolling(3).sum()

  for i in range(due_to_varb):
    power_bi['3MMT'+dt_col_names[i]+'_using_sales_index'] = Due_to['DT_'+dt_col_names[i]+'_using_sales_index'].rolling(3).sum()
  ##############################################################################################################
  for i in range(due_to_varb):
    power_bi[dt_col_names[i]+'6MMT'] = dt_raw[dt_col_names[i]].rolling(6).sum()

  for i in range(due_to_varb):
    power_bi['6MMT'+dt_col_names[i]+'_using_sales_index'] = Due_to['DT_'+dt_col_names[i]+'_using_sales_index'].rolling(6).sum()
  ##########################################################################################################
  ############ power bi melt--part ###############################################################################
  df_cat = pd.melt(power_bi, id_vars = ['States','Category', 'Date'], var_name='Metrics')
  # df_cat.head(4)

  #################################################################################################################
  ###################################################################################################################################
  ###########################################################################################################################################
  ###########################################################################################################################################
  estimatorts1 = estimatorts

  with pd.ExcelWriter('Juices_March_Results_{}.xlsx'.format(all_state[sta]),datetime_format  = 'YYYY-MM-DD') as writer:  
      coke_ols.to_excel(writer,    index=False, sheet_name = 'Ensemble')        
      estimatorts1.to_excel(writer,index=False, sheet_name = 'RF+ANN+OLS')          
      Due_to.to_excel(writer,      index=False, sheet_name = 'Due_to')          
      power_bi.to_excel(writer,    index=False, sheet_name = 'bi')              
      df_cat.to_excel(writer,      index=False, sheet_name = 'Power_BI')
      

MAPE OLS: 9.571857255101543
------------------------------------------------------------------------------------------------------
Processing for state:  West Bengal
(59, 6)
(59, 1)
(3, 6)
(3, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


                           R_square for training data: 0.9953940100110832
                           R_square for testing data: 0.9655527197971489
                           MAPE for training data: 3.187974691173593
                           MAPE for testing data: 5.874204671710262
[0.00580795 0.00426355 0.00158656 0.00375236 0.59298299 0.3916066 ]
RF mape:- 3.2652876316567108
{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 1, 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': 256, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 50, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 1840, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                R_square for training data: 0.9727292190642672
                R_square for testing data: 0.5537017847395499
                MAPE for training data: 8.876203340929159
                MAPE for testing data: 10.757392562454845
ANN mape:- 8.824891663526744
MAPE Ensemble: 4.031610173660293
MAPE OLS: 7.703117197609423
------------------------------------------------------------------------------------------------------
Processing for state:  Delhi
(59, 8)
(59, 1)
(3, 8)
(3, 1)


...

                R_square for training data: 0.9236814723005096
                R_square for testing data: -1.2000952991835527
                MAPE for training data: 13.102791482315515
                MAPE for testing data: 19.706888465668644
ANN mape:- 13.209291473867005
MAPE Ensemble: 6.130380879092847
MAPE OLS: 8.565128303270077
------------------------------------------------------------------------------------------------------
Processing for state:  Tamil Nadu
(59, 7)
(59, 1)
(3, 7)
(3, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


                           R_square for training data: 0.9692157847547526
                           R_square for testing data: -1.0654974110211493
                           MAPE for training data: 2.523236249246232
                           MAPE for testing data: 35.110527396857364
[0.34030758 0.03878017 0.01358832 0.0397482  0.02424912 0.0127371
 0.53058951]
RF mape:- 4.034960649144441
{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 1, 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': 256, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 50, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 3380, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                R_square for training data: 0.7716614915050348
                R_square for testing data: -22.345171598736027
                MAPE for training data: 7.331674245658577
                MAPE for testing data: 30.273183264253316
ANN mape:- 8.30775127439073
MAPE Ensemble: 5.330592736294499


In [0]:
wb = pd.read_excel('Juices_March_Results_WB.xlsx',sheet_name='Power_BI') 
de = pd.read_excel('Juices_March_Results_DE.xlsx',sheet_name='Power_BI')  
mh = pd.read_excel('Juices_March_Results_MH.xlsx',sheet_name='Power_BI') 
ap = pd.read_excel('Juices_March_Results_AP.xlsx',sheet_name='Power_BI') 
up = pd.read_excel('Juices_March_Results_UP.xlsx',sheet_name='Power_BI') 
pu = pd.read_excel('Juices_March_Results_PU.xlsx',sheet_name='Power_BI') 
ka = pd.read_excel('Juices_March_Results_KA.xlsx',sheet_name='Power_BI') 
gu = pd.read_excel('Juices_March_Results_GU.xlsx',sheet_name='Power_BI') 
ra = pd.read_excel('Juices_March_Results_RA.xlsx',sheet_name='Power_BI') 
mp = pd.read_excel('Juices_March_Results_MP.xlsx',sheet_name='Power_BI') 
orr = pd.read_excel('Juices_March_Results_OR.xlsx',sheet_name='Power_BI') 
bi = pd.read_excel('Juices_March_Results_BI.xlsx',sheet_name='Power_BI') 
ch = pd.read_excel('Juices_March_Results_CH.xlsx',sheet_name='Power_BI') 
ha = pd.read_excel('Juices_March_Results_HA.xlsx',sheet_name='Power_BI') 
tn = pd.read_excel('Juices_March_Results_TN.xlsx',sheet_name='Power_BI')

merge1=wb.append(de)
merge2=merge1.append(mh)
merge3=merge2.append(ap)
merge4=merge3.append(up)
merge5=merge4.append(pu)
merge6=merge5.append(ka)
merge7=merge6.append(gu)
merge8=merge7.append(ra)
merge9=merge8.append(mp)
merge10=merge9.append(orr)
merge11=merge10.append(bi)
merge12=merge11.append(ch)
merge13=merge12.append(ha)
merge14=merge13.append(tn)

merge14.to_excel('Juices_Data_Update_for_March_Power_BI_Consolidated_new.xlsx',index=False)